In [1]:
from superfv import OutputLoader

In [2]:
def get_total_time(sim, cat):
    times = [
        timer.data[cat].cum_time for timer in sim.minisnapshots["stepper_timer"][1:]
    ]
    return sum(times)


def get_times(sim, cats):
    take_step = get_total_time(sim, "take_step")
    return {cat: get_total_time(sim, cat) / take_step * 100 for cat in cats}

In [3]:
base_path = "/scratch/gpfs/jp7427/out/timing-of-2d-sine-wave/cupy/"

sim = OutputLoader(f"{base_path}ZS7/lazy/N_2048")

Successfully read simulation output from "/scratch/gpfs/jp7427/out/timing-of-2d-sine-wave/cupy/ZS7/lazy/N_2048"


In [4]:
cats = [
    "compute_dt",
    "update_workspaces",
    "interpolate_faces",
    "integrate_fluxes",
    "compute_RHS",
    "zhang_shu_limiter",
    "update_workspaces:shock_detector",
    "slope_limiter:detect_smooth_extrema",
    "integrate_fluxes:fallback",
    "integrate_fluxes:riemann_solver",
    "integrate_fluxes:quadrature",
]

times = get_times(sim, cats)

for cat, time in times.items():
    print(f"{cat:36s}: {times[cat]:5.2f} %")

compute_dt                          :  0.66 %
update_workspaces                   :  3.04 %
interpolate_faces                   : 43.46 %
integrate_fluxes                    : 15.77 %
compute_RHS                         :  1.50 %
zhang_shu_limiter                   : 32.87 %
update_workspaces:shock_detector    :  0.00 %
slope_limiter:detect_smooth_extrema :  0.00 %
integrate_fluxes:fallback           :  9.68 %
integrate_fluxes:riemann_solver     :  4.43 %
integrate_fluxes:quadrature         :  1.10 %
